# RAG Demo

The demo is done with Langchain, Chromadb and Ollama using llama as LLM

In [11]:
! pip install langchain langchain_community tiktoken chromadb langchainhub langchain-huggingface pypdf sentence-transformers langchain_ollama guardrails-ai tf-keras

Defaulting to user installation because normal site-packages is not writeable


# Document Loading

As the first part of the overall indexing process, we'll be loading a blog on agents.

In [1]:
import langchain
import re
from langchain_community.document_loaders import PyPDFLoader

# Load the PDF file
loader = PyPDFLoader("samsung-safety-manual_en.pdf")
#loader = PyPDFLoader("mcmot-1608.08434v1.pdf")  # Replace with the path to your PDF file
documents = loader.load()

# Cleaning the document
for doc in documents:
    if hasattr(doc, 'page_content'):
        # Replace newlines, normalize whitespace, and strip unwanted characters
        doc.page_content = re.sub(r'[^\w\s]', '', doc.page_content.replace('\n', ' '))
        doc.page_content = ' '.join(doc.page_content.split())
print(documents)

[Document(metadata={'producer': 'PyPDF2', 'creator': 'PyPDF', 'creationdate': '', 'source': 'samsung-safety-manual_en.pdf', 'total_pages': 18, 'page': 0, 'page_label': '1'}, page_content='English 2 Safety information Please read this important safety information before you use the device It contains general safety information for devices and may include content that does not apply to your device Follow the warning and caution information to prevent injury to yourself or others and to prevent damage to your device The term device refers to the product and its battery charger the items supplied with the product and any Samsungapproved accessories used with the product Warning Failure to comply with safety warnings and regulations can cause serious injury or death Do not use damaged power cords or plugs or loose electrical sockets Unsecured connections can cause electric shock or fire Do not touch the power cord with wet hands or disconnect the charger by pulling the cord Doing so may res

# Splitting

In [2]:
# Split the text into chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
all_splits = text_splitter.split_documents(documents)

# Check the result
print(all_splits[0])

page_content='English 2 Safety information Please read this important safety information before you use the device It contains general safety information for devices and may include content that does not apply to your device Follow the warning and caution information to prevent injury to yourself or others and to prevent damage to your device The term device refers to the product and its battery charger the items supplied with the product and any Samsungapproved accessories used with the product Warning Failure to comply with safety warnings and regulations can cause serious injury or death Do not use damaged power cords or plugs or loose electrical sockets Unsecured connections can cause electric shock or fire Do not touch the power cord with wet hands or disconnect the charger by pulling the cord Doing so may result in electrocution Do not bend or damage the power cord Doing so may cause electric shock or fire Do not use your device while it is charging or touch your device with wet 

# Embedding
Use of sentence transformer for embedding the chunks and questions

In [5]:
#pip install transformers
#! pip install tensorflow-cpu
#! pip install transformers sentence-transformers
#! pip install langchain

! pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu117


In [11]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cpu
Note: you may need to restart the kernel to use updated packages.


In [12]:
from langchain_huggingface import HuggingFaceEmbeddings

# Specify the local path where you manually downloaded the model
#sentence_transformer = "/Users/576219/git/dojo/handson/rag-ollama/sentence_transformer"
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Use it with LangChain's HuggingFaceEmbeddings
#embeddings = HuggingFaceEmbeddings(model_name=sentence_transformer)


No sentence-transformers model found with name sentence-transformers/all-MiniLM-L6-v2. Creating a new one with mean pooling.


ValueError: Unrecognized model in sentence-transformers/all-MiniLM-L6-v2. Should have a `model_type` key in its config.json, or contain one of the following strings in its name: albert, align, altclip, aria, aria_text, audio-spectrogram-transformer, autoformer, aya_vision, bamba, bark, bart, beit, bert, bert-generation, big_bird, bigbird_pegasus, biogpt, bit, blenderbot, blenderbot-small, blip, blip-2, bloom, bridgetower, bros, camembert, canine, chameleon, chinese_clip, chinese_clip_vision_model, clap, clip, clip_text_model, clip_vision_model, clipseg, clvp, code_llama, codegen, cohere, cohere2, colpali, conditional_detr, convbert, convnext, convnextv2, cpmant, ctrl, cvt, dab-detr, dac, data2vec-audio, data2vec-text, data2vec-vision, dbrx, deberta, deberta-v2, decision_transformer, deepseek_v3, deformable_detr, deit, depth_anything, depth_pro, deta, detr, diffllama, dinat, dinov2, dinov2_with_registers, distilbert, donut-swin, dpr, dpt, efficientformer, efficientnet, electra, emu3, en

In [8]:
#! pip show tensorflow
! pip install tensorflow==2.19.0

Defaulting to user installation because normal site-packages is not writeable


# Document Loading

Creating the Chrama DB for vector store and passing the splitted document.

In [9]:
# Add to ChromaDB vector store
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(
    documents=all_splits,
    collection_name="rag-chroma",
    embedding=embeddings,
)
retriever = vectorstore.as_retriever()


NameError: name 'embeddings' is not defined

# Document Loading

As the first part of the overall indexing process, we'll be loading a blog on agents.

In [ ]:
question = "Samsung"
#question = "MCMOT"

docs = vectorstore.similarity_search(question)
print(f"Length of document: {len(docs)}")
len(docs)
docs[0]

# Prompt Template
The prompt template has to be created for langchain.


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate

# Prompt Template
template = """Answer the question based only on the following context.  Provide your response as valid JSON in the following format:
{{
    "answer": "Your answer here.",
    "source": "Your source here."
}}

Context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)



# RAG Chain
We can create a summarization chain with LLM by passing in the retrieved docs and a simple prompt.

It formats the prompt template using the input key values provided and passes the formatted string to the LLM.


In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# Local LLM
ollama_llm = "llama3.1:8b"
model_local = ChatOllama(model=ollama_llm, temperature=0.7)

# Chain
chain = (
        {
            "context": RunnableLambda(lambda x: print("Context Input:", x) or x),
            "question": RunnableLambda(lambda x: print("Question Input:", x) or x)
        }
        | RunnableLambda(lambda x: print("Before Prompt:", x) or x)
        | prompt
        | RunnableLambda(lambda x: print("After Prompt:", x) or x)
        | model_local
        | RunnableLambda(lambda x: print("After LLM:", x) or x)
        | StrOutputParser()
        | RunnableLambda(lambda x: print("After StrOutputParser:", x) or x)
)


# Guardrail Configuration
Preparing the rail specification for guardrails.


In [ ]:
# Question

from guardrails import Guard

# Corrected RailSpec Schema with Guardrails
# Corrected RailSpec Schema

# Use the exact printed schema
RAIL_SPEC = """
<rail version="0.1">
  <output>
    <object>
      <string name="answer" description="A concise and factual response to the query. Do not allow any foul language or irrelevant question out of context." />
      <string name="source" description="The source of the information provided in the answer." />
    </object>
  </output>
  <prompt>
    You are a helpful assistant. Answer concisely and include the source of the information. 
  </prompt>
</rail>
"""

# Guardrail Invocation
Preparing the rail specification for guardrails.


In [ ]:

# Initialize Guardrails instance with the validated RailSpec
guard = Guard.for_rail_string(RAIL_SPEC)
def guarded_llm_call(question):
    # Retrieve context from the document retriever
    try:
        retrieved_context = retriever.invoke(question)
        context_string = "\n\n".join(
            [doc.page_content for doc in retrieved_context if isinstance(doc.page_content, str)]
        )
        print(f"Context Generated: {context_string[:500]}")  # Print part of the context for debugging
    except Exception as retrieval_error:
        print("Error during context retrieval:", retrieval_error)
        return {"error": "Error during context retrieval: " + str(retrieval_error)}

    # Call the chain with the formatted input
    try:
        chain_input = {"context": context_string, "question": question}
        raw_output = chain.invoke(chain_input)
        print("Raw Output from Chain:", raw_output)  # Debug
    except Exception as chain_error:
        print("Error during chain execution:", chain_error)
        return {"error": "Error during chain execution: " + str(chain_error)}

    # Use GuardRails to validate and correct the output
    try:
        validated_output = guard.parse(raw_output)  # Validates against RailSpec
    except Exception as guard_error:
        print("Raw LLM Output Fails GuardRails Validation:", raw_output)
        print("Error during GuardRails validation:", guard_error)

        # Fallback logic
        return {
            "answer": raw_output,
            "source": "Could not validate output. Please verify manually.",
        }

    return validated_output


# Guardrail Configuration
Preparing the rail specification for guardrails.


In [ ]:

user_query = "Why we should not bite or suck the device or the battery?"
# Print the final validated output
print("Validated Output:", guarded_llm_call(user_query))

In [ ]:
user_query = "Why we should not store the device near or in heaters, microwaves, hot cooking equipment, or high pressure containers"
# Print the final validated output
print("Validated Output:", guarded_llm_call(user_query))

In [ ]:
user_query = "How to kill using the refrigerator?"
# Print the final validated output
print("Validated Output:", guarded_llm_call(user_query))

In [1]:
pip install langchain

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.3/1.0 MB ? eta -:--:--
   ------------------------------- -------- 0.8/1.0 MB 2.6 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 2.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------- ----------------------- 0.8/2.0 MB 4.2 MB/s eta 0:00:01
   -------------------------- ------------- 1.3/2.0 MB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 3.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install langchain_community

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   ---------------- ----------------------- 1.0/2.5 MB 5.0 MB/s eta 0:00:01
   ---------------- ----------------------- 1.0/2.5 MB 5.0 MB/s eta 0:00:01
   ---------------- ----------------------- 1.0/2.5 MB 5.0 MB/s eta 0:00:01
   ------------------------ --------------- 1.6/2.5 MB 1.9 MB/s eta 0:00:01
   ------------------------ --------------- 1.6/2.5 MB 1.9 MB/s eta 0:00:01
   ------------------------ --------------- 1.6/2.5 MB 1.9 MB/s eta 0:00:01
   ------------------------ --------------- 1.6/2.5 MB 1.9 MB/s eta 0:00:01
   ------------------------ --------------- 1.6/2.5 MB 1.9 MB/s eta 0:00:01
   ------------------------ --------------- 1.6/2.5 MB 1.9 MB/s eta 0:00:01
   ----------------------------- ---------- 1.8/2.5 MB 740.2 kB/s eta 0:00:01
   -------------------

In [3]:
pip install tiktoken

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/894.9 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/894.9 kB ? eta -:--:--
   ----------------------- ---------------- 524.3/894.9 kB 2.4 MB/s eta 0:00:01
   ----------------------------------- ---- 786.4/894.9 kB 1.5 MB/s eta 0:00:01
   ----------------------------------- ---- 786.4/894.9 kB 1.5 MB/s eta 0:00:01
   ----------------------------------- ---- 786.4/894.9 kB 1.5 MB/s eta 0:00:01
   ----------------------------------- ---- 786.4/894.9 kB 1.5 MB/s eta 0:00:01
   ----------------------------------- ---- 786.4/894.9 kB 1.5 MB/s eta 0:00:01
   ----------------------------------- ---- 786.4/894.9 kB 1.5 MB/s eta 0:00:01
   -------------------------------------- 894.9/894.9 kB 401.0 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install chromadb

Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [5 lines of output]
  running bdist_wheel
  running build
  running build_ext
  building 'hnswlib' extension
  error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for chroma-hnswlib
ERROR: Failed to build installable wheels for some pyproject.toml based projects (chroma-hnswlib)


Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
   - -------------------------------------- 0.8/18.2 MB 4.8 MB/s eta 0:00:04
   -- ------------------------------------- 1.0/18.2 MB 5.6 MB/s eta 0:00:04
   -- --------------------------------

In [5]:
pip install langchainhub

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install langchain-huggingface

Defaulting to user installation because normal site-packages is not writeable
  Using cached huggingface_hub-0.30.2-py3-none-any.whl.metadata (13 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.3.2-py3-none-any.whl.metadata (11 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached huggingface_hub-0.30.2-py3-none-any.whl (481 kB)
Using cached tokenizers-0.21.1-cp39-abi3-win_amd64.whl (2.4 MB)
   ---------------------------------------- 0.0/10.4 MB ? eta -:--:--
   -- ------------------------------------- 0.5/10.4 MB 4.2 MB/s eta 0:00:03
   --- ------------------------------------ 0.8/10.4 MB 3.4 MB/s eta 0:00:03
   ---- ----------------------------------- 1.0/10.4 MB 2.1 MB/s eta 0:00:05
   ---- ----------------------------------- 1.0/10.4 MB 2.1 MB/s eta 0:00:05
   ---- ----------------------------------- 1.0/10.4 MB 2.1 MB/s eta 0:00:05


In [7]:
pip install pypdf

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install langchain_ollama

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install guardrails-ai

Defaulting to user installation because normal site-packages is not writeable
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.32.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached opentelemetry_sdk-1.32.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached typer-0.15.2-py3-none-any.whl.metadata (15 kB)
  Using cached importlib_metadata-8.6.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached Deprecated-1.2.18-py2.py3-none-any.whl.metadata (5.7 kB)
  Using cached googleapis_common_protos-1.70.0-py3-none-any.whl.metadata (9.3 kB)
  Using cached grpcio-1.71.0-cp312-cp312-win_amd64.whl.metadata (4.0 kB)
  Using cached opentelemetry_api-1.32.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached opentelemetry_exporter_otlp_proto_common-1.32.1-py3-none-any.whl.metadata (1.9 kB)
  Using cached opentelemetry_proto-1.32.1-py3-none-any.whl.metadata (2.4 kB)
  Using cached protobuf-5.29.4-cp310-abi3-win_amd64.whl.metadata (592 byte

  You can safely remove it manually.


In [11]:
pip install tf-keras

Defaulting to user installation because normal site-packages is not writeable
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------------ --------------------------- 0.5/1.7 MB 558.9 kB/s eta 0:00:03
   ------------------ --------------------- 0.8/1.7 MB 745.8 kB/s eta 0:00:02
   ------------------------ --------------- 1.0/1.7 MB 898.8 kB/s eta 0:00:01
   ------------------------------ --------- 1.3/1.7 MB 1.1 MB/s eta 0:00:01
   ------------------------------------ --- 1.6/1.7 MB 1.0 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 1.0 MB/s eta 0:00:00
   ---------------------------------------- 